In [ ]:
# pip install -U google-genai
import os, json, statistics
from collections import defaultdict
from google import genai
from google.genai import types

In [ ]:
MODEL = "gemini-2.5-flash" 
client = genai.Client()  # reads GEMINI_API_KEY from env

SAFETY = [
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
        threshold=types.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    ),
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_HARASSMENT,
        threshold=types.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    ),
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=types.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    ),
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_HARMFUL_CONTENT,
        threshold=types.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    ),
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
        threshold=types.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    ),
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_VIOLENCE,
        threshold=types.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    ),

]

In [ ]:
def build_prompt(frame_text, l3_labels_s, l2_labels_s, l1_labels_s, l3_labels_d, l2_labels_d, l1_labels_d):
    return f"""You are an expert judge for value relevance in misogyny frames.

Task: For EACH candidate label at EACH level, rate how strongly it applies to the frame.  
You will be given a rationale for each candidate value label. You must also take into account how sound the rationale is while scoring a candidate label.
Your task is to assess the relevance of the candidate labels to the frame, not to evaluate the frame itself.
Your final score should be a single number from 0 to 10, where 0 means the label is completely unrelated to the frame, and 10 means the label is highly relevant to the frame.
You will be given values both supported and disregarded by the frame, so you must evaluate the relevance of each value independently.

Frame:
{frame_text}

Values with SUPPORT stance:
- level_3 (top): {json.dumps(l3_labels_s, ensure_ascii=False)}
- level_2 (mid): {json.dumps(l2_labels_s, ensure_ascii=False)}
- level_1 (leaf): {json.dumps(l1_labels_s, ensure_ascii=False)}

Values with DISREGARD stance:
- level_3 (top): {json.dumps(l3_labels_d, ensure_ascii=False)}
- level_2 (mid): {json.dumps(l2_labels_d, ensure_ascii=False)}
- level_1 (leaf): {json.dumps(l1_labels_d, ensure_ascii=False)}

Return JSON ONLY with this shape:
{{
  "level_1": [{{"label": "str", "score_0_100": 0, "stance": "support|contradict|unclear", "rationale": "str"}}],
  "level_2": [{{"label": "str", "score_0_100": 0, "stance": "support|contradict|unclear", "rationale": "str"}}],
  "level_3": [{{"label": "str", "score_0_100": 0, "stance": "support|contradict|unclear", "rationale": "str"}}]
}}

Constraints:
- Score EVERY provided label at that level (no additions/removals).
"""

def judge_list_once(frame_text, l3_labels, l2_labels, l1_labels, seed=0, temperature=0.5):
    prompt = build_prompt(frame_text, l3_labels, l2_labels, l1_labels)
    resp = client.models.generate_content(
        model=MODEL,
        contents=prompt,
        config=types.GenerateContentConfig(
            response_mime_type="application/json",
            temperature=temperature,
            seed=seed,
            safety_settings=SAFETY,
            response_schema={
            "type":"object",
            "properties":{
                "id":{"type":"string"},
                "judgements":{
                    "type":"object",
                    "properties":{
                        "level_1_values":{"type":"object","additionalProperties":{"type":"number"}},
                        "level_2_values":{"type":"object","additionalProperties":{"type":"number"}},
                        "level_3_values":{"type":"object","additionalProperties":{"type":"number"}}
                    },
                    "required":["level_1_values","level_2_values","level_3_values"],
                    "additionalProperties": False
                }
            },
            "required":["id","judgements"],
            "additionalProperties": False
        }
    )
 ),

    return json.loads(resp.text)

def aggregate_replications(rep_outputs, levels=("level_1","level_2","level_3")):
    out = {}
    for lvl in levels:
        # collect per-label scores across replications
        scores_by_label = defaultdict(list)
        stances_by_label = defaultdict(list)
        rationales_by_label = defaultdict(list)
        for r in rep_outputs:
            for item in r[lvl]:
                scores_by_label[item["label"]].append(item["score_0_100"])
                stances_by_label[item["label"]].append(item["stance"])
                rationales_by_label[item["label"]].append(item["rationale"])
        # summarize
        merged = []
        for label, scores in scores_by_label.items():
            mean = sum(scores) / len(scores)
            stdev = statistics.pstdev(scores) if len(scores) > 1 else 0.0
            # majority stance (ties -> "unclear")
            counts = {s: stances_by_label[label].count(s) for s in ["support","contradict","unclear"]}
            maj = max(counts, key=counts.get)
            if len({counts[maj], *counts.values()}) > 1 and list(counts.values()).count(counts[maj]) > 1:
                maj = "unclear"
            merged.append({
                "label": label,
                "score_mean_0_100": round(mean, 2),
                "score_stdev": round(stdev, 2),
                "stance_majority": maj,
                "rationales": rationales_by_label[label],
                "rep_scores": scores,
            })
        # sort by mean score (desc), add rank
        merged.sort(key=lambda x: x["score_mean_0_100"], reverse=True)
        for i, m in enumerate(merged, start=1):
            m["rank"] = i
        out[lvl] = merged
    return out

def judge_list(frame_text, l3_labels, l2_labels, l1_labels, n=3):
    reps = [judge_list_once(frame_text, l3_labels, l2_labels, l1_labels, seed=i) for i in range(n)]
    return aggregate_replications(reps)

# --- example ---
if __name__ == "__main__":
    os.environ.setdefault("GEMINI_API_KEY", "<YOUR_API_KEY>")
    frame = "She only got the job because of quotas; real merit doesn’t matter."
    l1 = ["Merit/Desert", "Fairness/Equality", "Authority/Respect"]
    l2 = ["Meritocracy vs. Equality", "Social Hierarchy", "Workplace Competence"]
    l3 = ["Ideology & Social Order", "Gender Norms"]
    result = judge_list(frame, l3, l2, l1, n=3)
    # result["level_1"] / ["level_2"] / ["level_3"] → ranked arrays with mean±stdev
    print(json.dumps(result["level_1"][:3], ensure_ascii=False, indent=2))


In [1]:
from collections import defaultdict
import os
import json

def read_jsonl(path):
    with open(path, "r") as f:
        for line in f:
            line = line.strip()
            if line:
                ex = json.loads(line)
                yield ex

def write_jsonl(path, data):
    with open(path, "w") as f:
        for ex in data:
            f.write(json.dumps(ex) + "\n")

level3 = defaultdict(list)
level2 = defaultdict(list)
level1 = defaultdict(list)

for f in os.listdir('final_frames'):
    val_file = os.path.join('final_frames', f, 'values.jsonl')
    val_cont_file = os.path.join('final_frames', f, 'values_contra.jsonl')
    for line in read_jsonl(val_file):
        for val in line['response']['values']['level_1_values']:
            level3[val['level_1_value']].append(line['id'])
        for val in line['response']['values']['level_2_values']:
            level2[val['level_2_value']].append(line['id'])
        for val in line['response']['values']['level_3_values']:
            level1[val['level_3_value']].append(line['id'])
    for line in read_jsonl(val_cont_file):
        for val in line['response']['values']['level_1_values']:
            level3[val['level_1_value']].append(line['id'])
        for val in line['response']['values']['level_2_values']:
            level2[val['level_2_value']].append(line['id'])
        for val in line['response']['values']['level_3_values']:
            level1[val['level_3_value']].append(line['id'])

In [5]:
level3.keys()

dict_keys(['Self-Enhancement', 'Self-Transcendence', 'Conservation', 'Openness to Change', 'Hedonism'])

In [7]:
len(level2.keys())

19

In [8]:
len(level1.keys())

127

In [2]:
actual_level1=["Be creative","Be curious",
    "Have freedom of thought",
                "Be choosing own goals",
                "Be independent",
                "Have freedom of action",
                "Have privacy",
                "Have an exciting life",
                "Have a varied life",
                "Be daring",
                "Have pleasure",
                "Be ambitious",
                "Have success",
                "Be capable",
                "Be intellectual",
                "Be courageous",
                "Have influence",
                "Have the right to command",
                "Have wealth",
                "Have social recognition",
                "Have a good reputation",
                "Have a sense of belonging",
                "Have good health",
                "Have no debts",
                "Be neat and tidy",
                "Have a comfortable life",
                "Have a safe country",
                "Have a stable society",
                "Be respecting traditions",
                "Be holding religious faith",
                "Be compliant",
                "Be self-disciplined",
                "Be behaving properly",
                "Be polite",
                "Be honoring elders",
                "Be humble",
                "Have life accepted as is",
                "Be helpful",
                "Be honest",
                "Be forgiving",
                "Have the own family secured",
                "Be loving",
                "Be responsible",
                "Have loyalty towards friends",
                "Have equality",
                "Be just",
                "Have a world at peace",
                "Be protecting the environment",
                "Have harmony with nature",
                "Have a world of beauty",
                "Be broadminded",
                "Have the wisdom to accept others",
                "Be logical",
                "Have an objective view"
            ]

In [23]:
len(actual_level1)

54

In [3]:
keys_to_delete =[k for k in level1 if k not in actual_level1]
for k in keys_to_delete:
    del level1[k]

In [4]:
len(level1.keys())

51

In [5]:
print(actual_level1-level1.keys())

{'Be protecting the environment', 'Have harmony with nature', 'Have a world of beauty'}


In [ ]:
import json 
level1_frame_count={k: len(v) for k, v in level1.items()}
print(json.dumps(level1_frame_count, indent=2))

{
  "Have the right to command": 562,
  "Have social recognition": 131,
  "Have a good reputation": 155,
  "Have influence": 209,
  "Be intellectual": 31,
  "Be loving": 526,
  "Have equality": 1044,
  "Be just": 1000,
  "Be broadminded": 894,
  "Have the wisdom to accept others": 505,
  "Be respecting traditions": 666,
  "Be helpful": 272,
  "Be curious": 188,
  "Have freedom of thought": 771,
  "Be independent": 683,
  "Have freedom of action": 664,
  "Have a world at peace": 27,
  "Have a safe country": 31,
  "Have a stable society": 228,
  "Be logical": 225,
  "Have pleasure": 213,
  "Be self-disciplined": 75,
  "Be behaving properly": 259,
  "Have wealth": 74,
  "Be compliant": 255,
  "Have good health": 73,
  "Have a comfortable life": 32,
  "Have a sense of belonging": 30,
  "Be honoring elders": 13,
  "Be choosing own goals": 338,
  "Have privacy": 111,
  "Be polite": 173,
  "Have the own family secured": 18,
  "Be creative": 64,
  "Have an exciting life": 30,
  "Have a varied 